In [1]:
import pandas as pd 
import numpy as np 

print("Hello Capstone Project Course")

Hello Capstone Project Course


This is for Segmenting and Clustering Neighborhoods in Toronto

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import bs4

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, "html.parser")
#tbody = soup.find_all(id='packages_list')[0].tbody

#for tr in tbody.find_all('tr'):
#    package = tr.find_all('td')[0].contents[2].strip()
#    version = tr.find_all('td')[1].contents[0].strip()
#    print('{0} - {1}'.format(package, version))

#table = soup.find("table", id="packages_list")
row_data = []
#for row in table.find_all("tr"):
#    cols = row.find_all("td")
#    cols = [ele.text.strip() for ele in cols]
#    row_data.append(cols)

contentTable  = soup.find('table', { "class" : "wikitable sortable"}) # Use dictionary to pass key : value pair
rows  = contentTable.find_all('tr')
for row in rows:
    #print(row.get_text())
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    row_data.append(cols)

In [37]:
df = pd.DataFrame(row_data)
df.columns = ['PostalCode', 'Borough','Neighbourhood']
df = df.iloc[1:]
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [38]:
df.tail(10)

,PostalCode,Borough,Neighbourhood
279,M4Z,Not assigned,Not assigned
280,M5Z,Not assigned,Not assigned
281,M6Z,Not assigned,Not assigned
282,M7Z,Not assigned,Not assigned
283,M8Z,Etobicoke,Kingsway Park South West
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


Ignore cells with a borough that is Not assigned.

In [39]:
df_clean =df[df['Borough']!='Not assigned']

df_clean.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [40]:
df_clean.tail(10)

,PostalCode,Borough,Neighbourhood
270,M8Y,Etobicoke,Mimico NE
271,M8Y,Etobicoke,Old Mill South
272,M8Y,Etobicoke,The Queensway East
273,M8Y,Etobicoke,Royal York South East
274,M8Y,Etobicoke,Sunnylea
283,M8Z,Etobicoke,Kingsway Park South West
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor


In [41]:
df_clean[df_clean['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
the value of the Borough and the Neighborhood columns will be Queen's Park.

In [42]:
#df_clean[df_clean.Neighbourhood=='Not assigned']['Neighbourhood'] = 'Queen''s Park'

df_clean.loc[ (df_clean.Neighbourhood=='Not assigned'),'Neighbourhood']= 'Queen''s Park'

#df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['B'] == 'square'),'label'] = 'M5'

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [44]:
df_clean[(df_clean.PostalCode=='M7A')]

,PostalCode,Borough,Neighbourhood
9,M7A,Queen's Park,Queens Park


In [45]:
df_clean.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queens Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [46]:
df_final = df_clean.groupby(['PostalCode','Borough'], sort=False).agg( ','.join)


In [47]:
df_final=df_final.reset_index()

In [48]:
df_final.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queens Park


In [49]:
df_final.shape

(103, 3)

Read postal code coordinates file 

In [50]:
url="https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"

df_post=pd.read_csv(url)

df_post.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
df_post.tail(10)

,Postal Code,Latitude,Longitude
93,M9A,43.667856,-79.532242
94,M9B,43.650943,-79.554724
95,M9C,43.643515,-79.577201
96,M9L,43.756303,-79.565963
97,M9M,43.724766,-79.532242
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [51]:
# use downloaded PostCode Lat and Long to update our neighbourhood 
# First change Postal Code to PostalCode
df_post.columns = ['PostalCode', 'Latitude','Longitude']

In [52]:
new_df =pd.merge(df_final, df_post, on='PostalCode', how='inner')

In [53]:
new_df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queens Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [54]:
new_df.tail(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
93,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
94,M9W,Etobicoke,Northwest,43.706748,-79.594054
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
96,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
97,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648429,-79.382280
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
